In [1]:
from git import Repo #To clone the repository when user provides a git URL
import os
from langchain.text_splitter import Language #To set the programming language for the code files when schunking the code files
from langchain.document_loaders.generic import GenericLoader
# from langchain.document_loaders import DirectoryLoader 
from langchain.document_loaders.parsers import LanguageParser 
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

# Create a file an prepare URL for cloning to the file

In [2]:
%pwd

'/Users/georgekuncheria/Desktop/GenAI/End-To-End_Projects/SourceCodeAnalysis/research'

In [3]:
!mkdir test_repo

In [4]:
repo_path = "test_repo/"
repo = Repo.clone_from("https://github.com/GeorgeKuncheria/MedicalChatBot.git", to_path=repo_path)

# Loading all the python files from the created Repository Path

In [5]:
# loader = GenericLoader.from_filesystem(
#     repo_path,
#     glob="**/*.py",
#     parser=LanguageParser(language=Language.PYTHON, parser_threshold=500),
# )

loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
                                        )

In [6]:
documents = loader.load()
len(documents)

8

# Create Text Chunks

In [7]:
documents_splitter= RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=500,
    chunk_overlap=20
)


In [8]:
texts =documents_splitter.split_documents(documents)

In [9]:
len(texts)

14

# Load and embed using OpenAI

In [10]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [11]:
embeddings = OpenAIEmbeddings(disallowed_special=())

# Create & Persist vector embeddings to ChromaDB

In [12]:
vectordb= Chroma.from_documents(texts,embedding=embeddings, persist_directory='./db')

Failed to send telemetry event client_start: capture() takes 1 positional argument but 3 were given


In [13]:
vectordb.persist()

# Creating OpenAI LLM

In [14]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI()

# Creating Chain

In [15]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [16]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

In [17]:
question = "what is download_hugging_face_embeddings funtion?"

In [19]:
result = qa(question)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 14, updating n_results = 14


The purpose of the `download_hugging_face_embeddings` function is to download embeddings from the Hugging Face model "sentence-transformers/all-MiniLM-L6-v2" for use in the application.


In [20]:
question = "what is load_pdf_file funtion?"
result = qa(question)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 14, updating n_results = 14


The purpose of the `load_pdf_file` function is to load PDF files from a specified directory using the `DirectoryLoader` and `PyPDFLoader` classes. It retrieves the documents from the directory and returns them for further processing or analysis within the application.
